### 5.1 二维卷积层

### 5.1.1 二维互相关运算

> **互相关运: 卷积核与对应位置的元素一一相乘后相加**

In [1]:
import torch
from torch import nn

In [7]:
def corr2D(X, K):
    h, w = K.shape
    Y = torch.zeros(X.shape[0] - h + 1, X.shape[1] - w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i][j] = (X[i: i+h, j:j+w] * K) .sum()
    return Y

In [8]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
Y = torch.tensor([[0, 1], [2, 3]])
corr2D(X, Y)

tensor([[19., 25.],
        [37., 43.]])

### 5.1.2 二维卷积层

In [21]:
class Conv2D(nn.Module):
    def __init__(self, kernal_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernal_size))
        self.bias = nn.Parameter(torch.randn(1))
    
    def forward(self, x):
        return corr2D(X, self.weight) + self.bias

### 5.1.3 图像中物体边缘检测

In [22]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [23]:
# 构造一个宽和高分别为1和2的卷积核K
# 如果横向相邻元素相同，则输出位0
K = torch.tensor([[1, -1]])
K.shape

torch.Size([1, 2])

In [24]:
# 卷积层可通过重复使用卷积核有效的表征局部空间
Y = corr2D(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

### 5.1.4 通过数据学习核数组

In [25]:
# 构造一个核数据形状是(1, 2)的二维卷积层
conv2d = Conv2D(kernal_size=(1, 2))

step = 20
lr = 0.01

for i in range(step):
    Y_hat = conv2d(X)
    loss = ((Y_hat - Y) ** 2).sum()
    loss.backward()
    
    # 梯度下降
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad
    
    # 梯度清零
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    
    if (i+1) % 5 == 0:
        print('Step %d, loss %.3f', (i+1, loss.item()))
    

Step %d, loss %.3f (5, 7.11964750289917)
Step %d, loss %.3f (10, 1.423753261566162)
Step %d, loss %.3f (15, 0.3341445028781891)
Step %d, loss %.3f (20, 0.08610104024410248)


In [27]:
# 此时参数与之前定义的核数组K较接近
print('weight', conv2d.weight.data)
print('bias', conv2d.bias.data)

weight tensor([[ 0.9178, -0.9372]])
bias tensor([0.0108])


### 5.1.5 互相关运算和卷积运算

> 1. **卷积运算: 将核数组左右翻转并上下翻转,再与输入的数组做`互相光运算`**
> 2. **卷积层无论使用 `卷积运算` 或 `互相关运算` 都不影响模型预测的输出**

### 5.1.6 特诊图和感受野

> 1. **`特征图(feature map)`: 二维卷积层输出的二维数组可以看做是输入在空间维度(宽和高)上某一级的表征**
> 2. **`感受野(receptive field)`: 影响元素x的钱箱计算的所有可能输入区域(可能大于输入的实际尺寸)**